In [19]:
import os

# Access the secret
my_openaikey = os.getenv("MY_OPENAIKEY")

if not my_openaikey:
    raise EnvironmentError("MY_OPENAIKEY environment variable is not set.")

# Set the OpenAI API key
os.environ['OPENAI_API_KEY'] = my_openaikey

In [4]:
%%capture
!pip install  langchain     #Langchain
!pip install  langchain-openai
!pip install -U langchain-community #This contains all the document loaders

!pip install -qU pypdf       #loader for pdfs
!pip install -q faiss-gpu   #FAISS vector database and similarity of embeddings
!pip install chromadb       #Vector database

#!pip install langchain-cohere  #Use this when you need to use cohere embeddings

!pip install tiktoken       #tokenizer for Open AI models
!pip install streamlit      #GUI for apps
!pip install PyPDF2

In [23]:
#A new llm function that will work with the free key provided to you
from langchain_openai import ChatOpenAI
def get_llm_with_free_802key(temperature, model):
  api_key_from_kyle_and_sudhir = os.getenv("MY_OPENAIKEY")
  return ChatOpenAI(
    api_key=api_key_from_kyle_and_sudhir,
    base_url="https://api.802.mba/api/providers/openai/v1/",
    model_name=model,
    temperature=temperature
  )
#How to call this function
llm = get_llm_with_free_802key(temperature=0.7, model="gpt-4o-mini")

In [24]:
%%writefile docchatapp.py
import os
import streamlit as st                        # used to create our UI frontend

#Libraries for Document Loaders
#from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFLoader # import loaders
from langchain_openai import ChatOpenAI

#Libraries for Document Splitting, embeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

#Libraries for VectorStores
from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS

#Chain for Q&A after retrieving external documents
from langchain.chains import RetrievalQA

def get_llm(temperature, model):
  my_openaikey=os.environ['OPENAI_API_KEY']
  return ChatOpenAI(
    api_key=my_openaikey,
    model_name=model,
    temperature=temperature
  )

#A new llm function that will work with the free key provided to you
from langchain_openai import ChatOpenAI
def get_llm_with_free_802key_1():
  api_key_from_kyle_and_sudhir = os.environ['OPENAI_API_KEY']
  return ChatOpenAI(
    api_key=api_key_from_kyle_and_sudhir,
    base_url="https://api.802.mba/api/providers/openai/v1/",
    model_name="gpt-4o-mini",
    temperature=0
  )
#Title for the StreamLit Page
st.title('Chat about the constitution')
# Loading documents (Make sure constitution.pdf is available in the  directory)
loader = PyPDFLoader("constitution.pdf")
documents = loader.load()
# print(documents) # print to ensure document loaded correctly.

#Splitting Documents into Chunks for embeddings and the store them in vector stores
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)
# to see the chunks
# st.write(chunks[0])
# st.write(chunks[1])
embeddings = OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])
vector_store = FAISS.from_documents(chunks, embeddings)
# initialize OpenAI instance and set up a chain for Q&A from an LLM
#llm=get_llm(temperature=0.7, model="gpt-3.5-turbo")
#llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)
llm = get_llm_with_free_802key_1()

retriever=vector_store.as_retriever()
chain = RetrievalQA.from_chain_type(llm, retriever=retriever)


# get question from user input using streamlit interface
#Example question: "What are the three branches of government?"
question = st.text_input('Input your question:')
if question:
  # run chain
  result = chain.invoke(question)
  response = result['result']
  st.write(response)

Overwriting docchatapp.py


In [25]:
#Installing local tunnel, getting the IP address, running the streamlit app, and running the app with local tunnel
!npm install localtunnel
!npm audit fix --force
! wget -q -O - ipv4.icanhazip.com
!streamlit run docchatapp.py &>/content/logs.txt
#get_ipython().system('streamlit run docchatapp.py')
! npx localtunnel --port 8501

⠙⠹⠸⠼⠴
up to date, audited 23 packages in 583ms
⠴
⠴3 packages are looking for funding
⠴  run `npm fund` for details
⠴
2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠴npm warn using --force Recommended protections disabled.
⠙⠹⠸npm warn audit Updating localtunnel to 1.8.3, which is a SemVer major change.
⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹npm warn deprecated cryptiles@2.0.5: This version has been deprecated in accordance with the hapi support policy (hapi.im/support). Please upgrade to the latest version to get the best features, bug fixes, and security patches. If you are unable to upgrade at this time, paid support is available for older versions (hapi.im/commercial).
⠹npm warn deprecated boom@2.10.1: This version has been deprecated in accordance with the hapi support policy (hapi.im/support). Please upgrade to the latest version to get the best features, bug fixes, and se